In [28]:
import pandas as pd
import numpy as np

In [ ]:
# %load functions/houseFunctions.py

In [41]:
parse_data("Datasets/1976-2016-house.csv")

KeyboardInterrupt: 

In [42]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
